# Inuk Song

## [1] data loading

In [1]:
# import mpdule
import pandas as pd
import numpy as np

In [2]:
# mount drive
import os
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
cd /content/drive/MyDrive/Colab Notebooks/Kaggle/default/data

/content/drive/MyDrive/Colab Notebooks/Kaggle/default/data


In [4]:
# load X data
X_train_1_raw = np.load("data_1.npy")
X_train_2_raw = np.load("data_2.npy")
X_train_3_raw = np.load("data_3.npy")
X_test_raw_1 = np.load("data_4.npy")

X_train_4_raw = np.load("data_5.npy")
X_train_5_raw = np.load("data_6.npy")
X_train_6_raw = np.load("data_7.npy")
X_test_raw_2 = np.load("data_8.npy")

In [5]:
# load y data
y_train_1_raw = pd.read_parquet("targets_1.pqt")
y_train_2_raw = pd.read_parquet("targets_2.pqt")
y_train_3_raw = pd.read_parquet("targets_3.pqt")
y_test_raw_1 = pd.read_parquet("targets_4.pqt")

y_train_4_raw = pd.read_parquet("targets_5.pqt")
y_train_5_raw = pd.read_parquet("targets_6.pqt")
y_train_6_raw = pd.read_parquet("targets_7.pqt")
y_test_raw_2 = pd.read_parquet("targets_8.pqt")

In [6]:
X_train_raw_a = np.concatenate([np.concatenate([X_train_1_raw, X_train_2_raw]), X_train_3_raw])
X_train_raw_b = np.concatenate([np.concatenate([X_train_4_raw, X_train_5_raw]), X_train_6_raw])
X_train_raw = np.concatenate([X_train_raw_a, X_train_raw_b])

X_test_raw = np.concatenate([X_test_raw_1, X_test_raw_2])

In [7]:
y_train_raw_a = pd.concat([pd.concat([y_train_1_raw, y_train_2_raw]), y_train_3_raw])
y_train_raw_b = pd.concat([pd.concat([y_train_4_raw, y_train_5_raw]), y_train_6_raw])
y_train_raw = pd.concat([y_train_raw_a, y_train_raw_b])

y_test_raw = pd.concat([y_test_raw_1, y_test_raw_2])

## [2] data preprocessing

In [8]:
# 변경 전 shape 출력
print(f"X_train_raw.shape: {X_train_raw.shape}, X_test_raw.shape: {X_test_raw.shape}")
print(f"y_train_raw.shape: {y_train_raw.shape}, y_test_raw.shape: {y_test_raw.shape}")

X_train_raw.shape: (275346, 13, 188), X_test_raw.shape: (91782, 13, 188)
y_train_raw.shape: (275346, 2), y_test_raw.shape: (91782, 2)


In [9]:
# X 데이터 3차원에서 2차원 데이터로 변경
X_train = X_train_raw.reshape(X_train_raw.shape[0], 13 * 188)
X_test = X_test_raw.reshape(X_test_raw.shape[0], 13 * 188)

In [10]:
# y 데이터 np array 타입으로 변경
y_train = np.array(y_train_raw["target"])
y_test = np.array(y_test_raw["target"])

In [11]:
# 변경 후 shape 출력
print(f"X_train.shape: {X_train.shape}, X_test.shape: {X_test.shape}")
print(f"y_train.shape: {y_train.shape}, y_test.shape: {y_test.shape}")

X_train.shape: (275346, 2444), X_test.shape: (91782, 2444)
y_train.shape: (275346,), y_test.shape: (91782,)


## [3] feature test

In [12]:
# 샘플링하기위한 샘플 데이터프레임 생성
X_select = pd.DataFrame(X_train_1_raw.reshape(X_train_1_raw.shape[0], 13 * 188))
y_select = y_train_1_raw["target"]

df_select = pd.concat([X_select, y_select], ignore_index=True, axis = 1)

In [13]:
# 샘플 데이터프레임에서 train 샘플 추출
df_select.sample(1600, random_state=42)

X_sample_train = df_select.iloc[:, :2444].values
y_sample_train = df_select[2444].values

# test 샘플 추출
df_select.sample(400, random_state=100)

X_sample_test = df_select.iloc[:, :2444].values
y_sample_test = df_select[2444].values

In [14]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

model_ETC = ExtraTreesClassifier()
model_ETC.fit(X_sample_train, y_sample_train)
print(model_ETC.feature_importances_)

[9.58761594e-05 1.93454681e-04 2.98474267e-04 ... 3.06076743e-04
 3.38116052e-04 2.85007423e-04]


In [15]:
print("sample train score: ", model_ETC.score(X_sample_train, y_sample_train))
print("sample test score: ", model_ETC.score(X_sample_test, y_sample_test))

print("train score: ", model_ETC.score(X_train, y_train))
print("test score: ", model_ETC.score(X_test, y_test))

sample train score:  1.0
sample test score:  1.0
train score:  0.9046254530663238
test score:  0.886873243119566


## [4] feature reduce

In [16]:
def columns_dict():
    
    columns = ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_63', 'D_64', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'D_66', 'B_20', 'D_68', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_29', 'B_30', 'S_18', 'D_86', 'D_87', 'R_17', 'R_18', 'D_88', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_22', 'R_23', 'D_91', 'D_92', 'D_93', 'D_94', 'R_24', 'R_25', 'D_96', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'B_36', 'B_37', 'R_26', 'R_27', 'B_38', 'D_108', 'D_109', 'D_110', 'D_111', 'B_39', 'D_112', 'B_40', 'S_27', 'D_113', 'D_114', 'D_115', 'D_116', 'D_117', 'D_118', 'D_119', 'D_120', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_127', 'D_128', 'D_129', 'B_41', 'B_42', 'D_130', 'D_131', 'D_132', 'D_133', 'R_28', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145']
    
    from collections import defaultdict
    legend = ["D", "S", "P", "B", "R"]
    # D_* = Delinquency variables
    # S_* = Spend variables
    # P_* = Payment variables
    # B_* = Balance variables
    # R_* = Risk variables
    
    result = defaultdict(list)
    
    for col in columns:
        if col[0] in legend:
            index = columns.index(col)
            result[col[0]].append(index)
            
    
    return result

In [17]:
def make_full_columns_dict(col_dict:dict, legend:list):
    from collections import defaultdict
    result = defaultdict()

    for i in legend:
        index_list = col_dict[i]
        base = np.array(col_dict[i])
        for j in range(1, 13):
            index_list = index_list + list(base + 188 * j)
        result[i] = index_list

    return result

In [18]:
col_dict = columns_dict()
legend = ["D", "S", "P", "B", "R"]

legend_index = make_full_columns_dict(col_dict, legend)

In [19]:
def make_mean_df(df:pd.DataFrame, legend_index:dict):
    from collections import defaultdict
    result = defaultdict()

    for i in legend_index.keys():
        result[i] = df.iloc[:, legend_index[i]].mean(axis=1)

    return pd.DataFrame(result)

In [20]:
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)
X_sample_train_df = pd.DataFrame(X_sample_train)
X_sample_test_df = pd.DataFrame(X_sample_test)


X_train_mean = make_mean_df(X_train_df, legend_index).values
X_test_mean = make_mean_df(X_test_df, legend_index).values
X_sample_train_mean = make_mean_df(X_sample_train_df, legend_index).values
X_sample_test_mean = make_mean_df(X_sample_test_df, legend_index).values

In [21]:
model_ETC_mean = ExtraTreesClassifier()
model_ETC_mean.fit(X_sample_train_mean, y_sample_train)
print(model_ETC_mean.feature_importances_)

[0.17247434 0.16909129 0.24669965 0.24597885 0.16575586]


In [22]:
print("sample train mean score: ", model_ETC_mean.score(X_sample_train_mean, y_sample_train))
print("sample test mean score: ", model_ETC_mean.score(X_sample_test_mean, y_sample_test))

print("train mean score: ", model_ETC_mean.score(X_train_mean, y_train))
print("test mean score: ", model_ETC_mean.score(X_test_mean, y_test))

sample train mean score:  1.0
sample test mean score:  1.0
train mean score:  0.8250092610751564
test mean score:  0.7913752151838052


## [5] model select

### [5-1] XGBoost 

In [28]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split

In [29]:
X_train_mean, X_val_mean, y_train, y_val = train_test_split(
    X_train_mean, y_train, stratify=y_train, random_state=42
)

In [ ]:
# # make model
# model_xgb = xgb.XGBRFClassifier(n_estimators=100)
# params = {'max_depth':[5,7], 'min_child_weight':[1,3], 'colsample_bytree':[0.5,0.75]}
# gridcv = GridSearchCV(model_xgb, param_grid=params, cv=3)

# gridcv.fit(X_train_mean, y_train, early_stopping_rounds=30, eval_metric='auc', eval_set=[(X_val_mean, y_val)])

In [33]:
print(gridcv.best_params_)

{'colsample_bytree': 0.75, 'max_depth': 7, 'min_child_weight': 1}


In [35]:
model_xgb = xgb.XGBClassifier(
    n_estimator=400,
    learning_rate=0.02,
    colsample_bytree=0.75,  # 각 트리마다 데이터 샘플링 비율
    max_depth=7,  # 트리의 최대 깊이
    min_child_weight=1,  # child에서 필요한 모든 관측치에 대한 가중치의 최소 
    reg_alpha=0.0  # 라쏘 가중치
)

In [ ]:
model_xgb.fit(
    X_train_mean,
    y_train,
    early_stopping_rounds=100,
    eval_metric='auc',
    eval_set=[(X_val_mean, y_val)]
)

In [37]:
model_xgb.score(X_train_mean, y_train)

0.7962945924874945

In [38]:
model_xgb.score(X_test_mean, y_test)

0.7903946307554858